<a href="https://colab.research.google.com/github/sherf1207/Amit-assignments/blob/main/seq2seq_attention_mechanism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Attention
from tensorflow.keras import layers
from tensorflow.keras.models import Model

In [ ]:
import pandas as pd
df=pd.read_csv('/english_french.csv')
df

,English,French
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !
...,...,...
229798,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
229799,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
229800,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...
229801,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...


In [ ]:
english_words=df['English']
french_words=df['French']

In [ ]:
# tokenizing the columns
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(english_words)
eng_sequences = eng_tokenizer.texts_to_sequences(english_words)
eng_vocab_size = len(eng_tokenizer.word_index) + 1

fr_tokenizer = Tokenizer()
fr_tokenizer.fit_on_texts(french_words)
fr_sequences = fr_tokenizer.texts_to_sequences(french_words)
fr_vocab_size = len(fr_tokenizer.word_index) + 1

In [ ]:
max_eng_length = max(len(word) for word in english_words)
max_fr_length = max(len(word) for word in french_words)

In [ ]:
eng_padded = pad_sequences(eng_sequences, maxlen=max_eng_length, padding='post')
fr_padded = pad_sequences(fr_sequences, maxlen=max_fr_length, padding='post')

In [ ]:
embedding_dim= 256
latent_dim = 256
encoder_inputs = layers.Input(shape=(max_eng_length,))
encoder_embedding = layers.Embedding(eng_vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = layers.LSTM(latent_dim, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]


decoder_inputs = layers.Input(shape=(max_fr_length,))
decoder_embedding = Embedding(fr_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

In [ ]:
attention_layer = Attention()
attention_out = attention_layer([decoder_outputs, encoder_outputs])

In [ ]:
# Concatenate attention output with decoder output
concate = tf.keras.layers.concatenate([decoder_outputs, attention_out])

In [ ]:
decoder_dense = Dense(fr_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(concate)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 290)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 349)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 290, 256)       │      4,257,792 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 349, 256)       │      8,980,736 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 256), (None,   │        525,312 │ embedding[0][0]        │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 349, 256),     │        525,312 │ embedding_1[0][0],     │
│                           │ (None, 256), (None,    │                │ lstm[0][1], lstm[0][2] │
│                           │ 256)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention (Attention)     │ (None, 349, 256)       │              0 │ lstm_1[0][0],          │
│                           │                        │                │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 349, 512)       │              0 │ lstm_1[0][0],          │
│                           │                        │                │ attention[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 349, 35081)     │     17,996,553 │ concatenate[0][0]      │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 32,285,705 (123.16 MB)

 Trainable params: 32,285,705 (123.16 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(
    [eng_padded, fr_padded[:, :-1]],
    fr_padded[:, 1:],
    epochs=20,
    batch_size=52)

Epoch 1/20


ValueError: Input 1 of layer "functional" is incompatible with the layer: expected shape=(None, 349), found shape=(None, 348)

### i've tried to do it with tensorflow but can't understand why it gives me an error but i will do it in pytorch ASAP